In [1]:
# 데이터 크롤링 모듈
from bs4 import BeautifulSoup
import re

In [2]:
# 데이터 분석 모듈
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
import requests

In [5]:
import requests
import json
cookies = {
    '_ga': 'GA1.2.930351172.1597235144',
    'Cookie_Id': 'didtlahs652',
    'ASP.NET_SessionId': 'hz0on0sdkdomw3nqzri3j4sk',
    '_gid': 'GA1.2.970217577.1599027608',
    '_gat': '1',
}

headers = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Whale/2.8.105.22 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://www.koreabaseball.com',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.koreabaseball.com/Schedule/Schedule.aspx',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}


In [6]:

list_columns = ["Month", "Date", "PlaceName", "HomeName", "AwayName", "HomeScore", "AwayScore", "IsHomeWin"]


save_date = None
df_match_days = pd.DataFrame(columns=list_columns)

for month in range(5, datetime.today().month + 1):

    data = {
      'leId': '1',
      'srIdList': '0,9',
      'seasonId': '2020',
        # 'gameMonth': "09",
      'gameMonth': str(month).zfill(2),
      'teamId': ''
    }

    # JSON
    response = requests.post('https://www.koreabaseball.com/ws/Schedule.asmx/GetScheduleList', headers=headers, cookies=cookies, data=data)
    json_data = json.loads(response.text)

    list_rows = json_data["rows"]

    for rows in list_rows:
        dic_add = {"Month":None, "Date": None, "PlaceName": None, "HomeName":None,
                   "AwayName" : None, "HomeScore":None, "AwayScore":None, "IsHomeWin" : None}
        rows = rows["row"]
        minus_idx = -1

        # 날짜 있는 데이터들 때문에 minus idx 해줌 행순서때문에.
        if rows[0]['Class'] == "day":
            save_date = rows[0]["Text"]
            minus_idx = 0
        # 우천취소시 데이터에 담지 않음 \ 그라운드사정
        if rows[8 + minus_idx]["Text"] != "-":
            continue

        # Month Day

        dic_add["Month"] = month
        dic_add["Date"] = save_date

        if month == datetime.today().month:
            data_day = int(save_date.split('.')[1][:2])
            real_today = datetime.today().today().day
            if data_day >=real_today :
                continue

        dic_add["PlaceName"] = rows[7 + minus_idx]["Text"]
        dic_add["HomeName"] = rows[2 + minus_idx]["Text"].lstrip("<span>").split("<")[0]
        dic_add["HomeName"] = rows[2 + minus_idx]["Text"].lstrip("<span>").split("<")[0]

        # MATCH STRING DATA
        match_data = rows[2 + minus_idx]["Text"].lstrip("<span>").rstrip("</span>")

        # SCORE
        score = re.findall('\d+', match_data)

        dic_add["HomeScore"] = int(score[1])
        dic_add["AwayScore"] = int(score[0])

        # HOME WIN ??
        if dic_add["HomeScore"] > dic_add["AwayScore"] :
            dic_add["IsHomeWin"] = 2
        elif dic_add["HomeScore"] == dic_add["AwayScore"] :
            # 비기면 1
            dic_add["IsHomeWin"] = 1
        else:
            dic_add["IsHomeWin"] = 0

        # CLUB NAME
        list_club_name = re.sub('<.*>', '.', match_data).split(".")
        dic_add["HomeName"] = list_club_name[1]
        dic_add["AwayName"] = list_club_name[0]

        df_match_days = df_match_days.append(dic_add, ignore_index=True)




df_match_days

,Month,Date,PlaceName,HomeName,AwayName,HomeScore,AwayScore,IsHomeWin
0,5,05.05(화),잠실,LG,두산,8,2,2
1,5,05.05(화),문학,SK,한화,0,3,0
2,5,05.05(화),대구,삼성,NC,0,4,0
3,5,05.05(화),수원,KT,롯데,2,7,0
4,5,05.05(화),광주,KIA,키움,2,11,0
...,...,...,...,...,...,...,...,...
473,9,09.01(화),잠실,두산,한화,4,0,2
474,9,09.01(화),문학,SK,LG,5,13,0
475,9,09.01(화),수원,KT,롯데,11,2,2
476,9,09.01(화),광주,KIA,삼성,6,0,2


In [9]:
df_match_days.to_csv(r'../Dataset/game_schedule.csv')